# Detección de Neumonía en Radiografías usando CNN
**Objetivo**: Entrenar una red neuronal convolucional para clasificar imágenes en NORMAL vs. PNEUMONIA.
**Dataset**: [Enlace al dataset de Kaggle](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data).

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import MobileNetV2

train_dir = 'data/train'
val_dir = 'data/validation'
test_dir = 'data/test'
img_width, img_height = 150, 150
batch_size = 32
epochs = 10

### Preprocesamiento de Imágenes
- **Data Augmentation**: Aplicamos transformaciones aleatorias para evitar sobreajuste.
- **Normalización**: Escalamos los valores de píxeles al rango [0, 1].
- **Clases detectadas**: El generador muestra cuántas imágenes hay en cada clase.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

### Arquitectura de la Red Neuronal Convolucional (CNN)

#### Capas Utilizadas  
1. **Capa Convolucional (`Conv2D`)**  
   - **Función**: Detecta características locales como bordes o texturas usando filtros (kernels).  
   - **Parámetros**:  
     - `32/64/128 filtros`: Aumenta progresivamente la complejidad de características detectadas.  
     - `Kernel (3,3)`: Tamaño del filtro (óptimo para imágenes de 150x150).  
     - `activation='relu'`: Introduce no linealidad (Rectified Linear Unit), evitando el problema de vanishing gradients.  

2. **Capa de Max Pooling (`MaxPooling2D`)**  
   - **Función**: Reduce la dimensionalidad espacial (resumen de características).  
   - **Parámetros**:  
     - `pool_size=(2,2)`: Reduce el mapa de características a la mitad (selecciona el valor máximo en ventanas de 2x2).  

3. **Capa Flatten**  
   - **Función**: "Aplana" la salida 3D de las capas convolucionales a 1D para conectarse a capas densas.  

4. **Capa Densa (`Dense`)**  
   - **Función**: Neuronas totalmente conectadas para clasificación.  
   - `512 unidades`: Suficiente capacidad para aprender patrones complejos.  
   - `Dropout(0.5)`: Apaga el 50% de las neuronas aleatoriamente durante el entrenamiento para prevenir overfitting.  

5. **Capa de Salida**  
   - `Dense(1, activation='sigmoid')`:  
     - **Sigmoid**: Ideal para clasificación binaria (devuelve probabilidad entre 0 y 1).  

#### Compilación del Modelo  
- **Optimizador Adam**:  
  - `learning_rate=0.0001`: Tasa de aprendizaje pequeña para ajustes finos.  
- **Pérdida (`binary_crossentropy`)**:  
  - Mide el error entre predicciones y etiquetas reales en problemas binarios.  
- **Métrica (`accuracy`)**:  
  - Porcentaje de predicciones correctas.  

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()  # Muestra la arquitectura

### Proceso de Entrenamiento  

#### Hiperparámetros Clave  
- `batch_size=32`: Compromiso entre velocidad y estabilidad del gradiente.  
- `epochs=10`: Número de pasadas completas sobre el dataset (puede ajustarse si hay underfitting/overfitting).  

#### Generadores de Datos  
- **`steps_per_epoch`**:  
  - Calculado como `total_muestras // batch_size`. Ejemplo: 5216 imágenes / 32 = 163 pasos/época.  
- **`validation_steps`**:  
  - Misma lógica para el dataset de validación.  

#### Monitoreo del Entrenamiento  
- **Historial (`history`)**:  
  - Almacena precisión y pérdida en entrenamiento/validación por época.  
  - **Uso**: Para graficar curvas y diagnosticar overfitting (si val_loss sube mientras train_loss baja).  

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)

model.save('modelo_neumonia.h5')

### Métricas de Rendimiento  

#### 1. Gráficas de Entrenamiento  
- **Exactitud (Accuracy)**:  
  - Si `val_accuracy` es mucho menor que `train_accuracy` → **Overfitting**.
- **Pérdida (Loss)**:  
  - Pérdida en validación debería disminuir establemente.

#### 2. Métricas con `sklearn.metrics`  
- **`classification_report`**:  
  - **Precision**: % de verdaderos positivos entre todos los positivos predichos.  
  - **Recall**: % de verdaderos positivos detectados correctamente.  
  - **F1-score**: Media armónica de precisión y recall (ideal para clases desbalanceadas).  
- **`confusion_matrix`**:  
  - Visualiza falsos positivos/negativos. Ejemplo:  
    ```
    [[TN FP]  
     [FN TP]]  
    ```  

In [ ]:
# Gráficas de precisión y pérdida
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Exactitud')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida')
plt.legend()
plt.show()

# Métricas con sklearn
test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int)

print("Informe de Clasificación:")
print(classification_report(test_generator.classes, predicted_classes))

print("Matriz de Confusión:")
print(confusion_matrix(test_generator.classes, predicted_classes))

### Transfer Learning con MobileNetV2  

#### ¿Por qué MobileNetV2?  
- **Ventajas**:  
  - Modelo preentrenado en ImageNet (aprovecha características genéricas).  
  - Arquitectura eficiente (menos parámetros que una CNN desde cero).  

#### Extracción de Características  
1. **`include_top=False`**:  
   - Elimina las capas densas finales de MobileNet para usar solo el extractor de características.  
2. **`pooling='avg'`**:  
   - Reduce la salida convolucional a un vector por imagen (Global Average Pooling).  

#### Random Forest  
- **Ventajas**:  
  - No requiere ajuste fino de hiperparámetros (vs. SVM o redes neuronales).  
  - Interpretabilidad (puede analizar importancia de características).  
- **Limitaciones**:  
  - Menos efectivo que redes neuronales si los datos son muy complejos (ej. imágenes de alta resolución).  

In [ ]:
# Extracción de características con MobileNetV2
mobilenet = MobileNetV2(
    input_shape=(img_width, img_height, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

# Generar características
train_features = mobilenet.predict(train_generator)
test_features = mobilenet.predict(test_generator)

# Entrenar Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_features, train_generator.classes)

# Evaluar
y_pred_rf = rf.predict(test_features)
print(classification_report(test_generator.classes, y_pred_rf))